In [27]:
import h5py
import numpy as np

np.random.seed(1337)  # for reproducibility

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, merge
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, ZeroPadding2D
from keras.regularizers import l2
from keras.utils import np_utils
from keras.optimizers import SGD, Adam, RMSprop
from keras import backend as K
from keras.engine.topology import Layer
from data_utils import *
import matplotlib.pyplot as plt

% matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
% load_ext autoreload
% autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
input_size = (3, 32, 32)
nb_classes = 10
# read data
f = h5py.File('cifar10_features.h5', 'r')
X_train_features = f['training_features'][:]
X_validation_features = f['validation_features'][:]
X_test_features = f['testing_features'][:]
f.close()
data = get_CIFAR10_data()
X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']
y_train = to_categorical(y_train, nb_classes)
y_val = to_categorical(y_val, nb_classes)

In [8]:
from transfer_learning_keras import classifier

In [40]:
model = classifier(l2_reg=1e-7, keep_prob=0.5)
model.load_weights('transfer_learning_cifar10.h5')
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-2, rho=0.9, epsilon=1e-08, decay=1e-6), 
              metrics=['accuracy'])
model.fit(X_train_features, y_train, batch_size=256, nb_epoch=100, verbose=1,
          validation_data=(X_validation_features, y_val))

score = np.mean((model.predict_classes(X_test_features, verbose=0) == y_test).astype(np.float32))
print 'Test accuracy', score

Train on 49000 samples, validate on 1000 samples
Epoch 1/100
11264/49000 [=====>........................] - ETA: 35s - loss: 2.2685 - acc: 0.6478

KeyboardInterrupt: 

In [39]:
model.save_weights('transfer_learning_cifar10.h5')